In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install torch

In [3]:
!pip install torch-summary

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary

In [5]:
alexnet = models.alexnet(pretrained=True)


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
type(alexnet)

torchvision.models.alexnet.AlexNet

In [7]:
list(alexnet.features)

[Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)]

In [8]:
features = models.alexnet().features  # get the feature extraction layers

In [9]:
features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
flatten = torch.nn.Flatten()# create a flatten layer
model = torch.nn.Sequential(features, flatten).to(device)  # create a new model with the flatten layer
summary(model)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0

In [11]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize((224,224)),                                #32*32 is very small for this model
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
# Create data loaders for batch processing
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64,shuffle=True, num_workers=2)
#val_loader = torch.utils.data.DataLoader(val_set, batch_size=len(val_set),shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64,shuffle=False, num_workers=2)

In [13]:
from tqdm import tqdm
def extract_features(loader):
    features = []
    labels = []
    for images, targets in tqdm(loader,desc = 'loading'):
        images = images.to(device)
        targets = targets.to(device)
        with torch.no_grad():
            outputs = model(images)
            features.extend(outputs.cpu().numpy())
            labels.extend(targets.cpu().numpy())
    return np.array(features), np.array(labels)


In [14]:
train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

loading: 100%|██████████| 157/157 [00:17<00:00,  9.09it/s]


In [15]:
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(test_labels.shape)

(50000, 9216)
(50000,)
(10000, 9216)
(10000,)


In [16]:
# Step 3: Train a new classification model using extracted features

from sklearn import svm
clf = svm.SVC(C=1.8, random_state=42,verbose =2)
clf.fit(train_features, train_labels)

[LibSVM]...*.*
optimization finished, #iter = 4396
obj = -4284.544255, rho = -2.285744
nSV = 3483, nBSV = 2739
...*.*
optimization finished, #iter = 4903
obj = -5714.854630, rho = -0.471291
nSV = 4221, nBSV = 3516
..*.*
optimization finished, #iter = 3721
obj = -4156.674780, rho = -1.114277
nSV = 3164, nBSV = 2601
...*.*
optimization finished, #iter = 4168
obj = -4743.687047, rho = -2.050528
nSV = 3566, nBSV = 2952
..*.*
optimization finished, #iter = 3631
obj = -3529.958927, rho = -1.428589
nSV = 2825, nBSV = 2227
..*
optimization finished, #iter = 2767
obj = -2936.398819, rho = -3.231896
nSV = 2318, nBSV = 1877
...*.*
optimization finished, #iter = 4112
obj = -3956.308627, rho = -2.450248
nSV = 3185, nBSV = 2513
....*..*
optimization finished, #iter = 6014
obj = -7323.259038, rho = -0.129188
nSV = 5415, nBSV = 4644
...*.*
optimization finished, #iter = 4376
obj = -4585.379572, rho = -2.119770
nSV = 3547, nBSV = 2841
..*.*
optimization finished, #iter = 3462
obj = -3128.612186, rho = 

SVC(C=1.8, random_state=42, verbose=2)

In [17]:
from sklearn.metrics import accuracy_score
pred_labels = clf.predict(test_features)
accuracy = accuracy_score(test_labels, pred_labels)
print('Accuracy:', accuracy)

Accuracy: 0.612


In [16]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier
xgbr = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
#xgbr = XGBClassifier(tree_method='gpu_hist',gpu_id=0,  min_child_weight = 5,learning_rate = 0.008,gamma = 0.3,colsample_bytree = 0.7)
xgbr.fit(train_features, train_labels)

In [19]:
# Step 4: Test classification accuracy
from sklearn.metrics import accuracy_score
pred_labels = xgbr.predict(test_features)
accuracy = accuracy_score(test_labels, pred_labels)
print('Accuracy:', accuracy)

Accuracy: 0.5666
